In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

ruta = "modelo/data/output/resultados.xlsx"

output_dir = "graficos_resultados"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print(f"Cargando datos desde: {ruta}")
print(f"Guardando gráficos en: {output_dir}/")

# Cargar las hojas del archivo Excel
try:
    df_gen = pd.read_excel(ruta, sheet_name="Generadores")
    df_scc = pd.read_excel(ruta, sheet_name="Cortocircuito")
    df_freq = pd.read_excel(ruta, sheet_name="Frecuencia")
except FileNotFoundError:
    print(f"Error: No se pudo encontrar el archivo en la ruta: {ruta}")
    print("Por favor, asegúrate de que el archivo exista en esa ubicación.")
    exit()
except Exception as e:
    print(f"Error al leer el archivo Excel: {e}")
    print("Asegúrate de que las hojas se llamen 'Generadores', 'Cortocircuito', y 'Frecuencia'.")
    exit()

print("Datos cargados exitosamente.")

# --- 2. Gráfico 1: Despacho de Generación (Stacked Bar) ---
print("Generando Gráfico 1: Despacho de Generación...")

# Pivotear los datos para tener el tiempo como índice y los generadores como columnas
try:
    # Asegurarnos de que los generadores sean categóricos para la leyenda
    df_gen['generator'] = 'Gen ' + df_gen['generator'].astype(str)
    
    gen_pivot = df_gen.pivot_table(index='time', columns='generator', values='p', aggfunc='sum')
    gen_pivot = gen_pivot.fillna(0) # Rellenar con 0 si un generador estaba apagado

    plt.figure(figsize=(14, 7))
    # Usamos gen_pivot.plot() directamente para que maneje la leyenda correctamente
    ax = gen_pivot.plot(kind='bar', stacked=True, figsize=(14, 7), width=0.8)
    
    ax.set_title('Despacho de Generación por Hora', fontsize=16)
    ax.set_xlabel('Hora (time)', fontsize=12)
    ax.set_ylabel('Potencia (p) [MW]', fontsize=12)
    ax.legend(title='Generador', bbox_to_anchor=(1.02, 1), loc='upper left')
    plt.xticks(rotation=45)
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Ajustar los ticks del eje x para que coincidan con las horas (de 1 a 24)
    if gen_pivot.index.min() == 1:
        ax.set_xticks(range(0, 24))
        ax.set_xticklabels(range(1, 25))
    else:
        ax.set_xticklabels(gen_pivot.index)

    plt.tight_layout(rect=[0, 0, 0.88, 1]) # Dejar espacio para la leyenda
    
    plot_path_gen = os.path.join(output_dir, "despacho_generacion.png")
    plt.savefig(plot_path_gen)
    plt.close()
    print(f"Gráfico 1 guardado en: {plot_path_gen}")

except Exception as e:
    print(f"Error al generar Gráfico 1: {e}")

# --- 3. Gráfico 2: SCC Mínima del Sistema vs. Límite ---
print("Generando Gráfico 2: SCC Mínima del Sistema...")

try:
    # Agrupar por hora y encontrar la SCC total mínima en esa hora
    scc_min_por_hora = df_scc.groupby('time')['I_total'].min()
    
    limit = 2.0  # Valor fijo según la nueva restricción en el modelo

    plt.figure(figsize=(14, 7))
    ax = scc_min_por_hora.plot(kind='line', marker='o', label='SCC Mínima del Sistema ($I_{total}$)', color='dodgerblue', linewidth=2)
    
    ax.axhline(y=limit, color='red', linestyle='--', linewidth=2, label=f'Límite SCC ($I_limit$ = {limit} p.u.)')
    
    ax.set_title('Corriente de Cortocircuito (SCC) Mínima del Sistema por Hora', fontsize=16)
    ax.set_xlabel('Hora (time)', fontsize=12)
    ax.set_ylabel('SCC (I_total) [p.u.]', fontsize=12)
    ax.legend(fontsize=12)
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    times = scc_min_por_hora.index
    ax.set_xticks(ticks=times[::1])
    ax.set_xticklabels([str(t) for t in times[::1]])
    ax.set_xlim(times.min(), times.max())
    
    plt.tight_layout()
    
    plot_path_scc = os.path.join(output_dir, "scc_minima_sistema.png")
    plt.savefig(plot_path_scc)
    plt.close()
    print(f"Gráfico 2 guardado en: {plot_path_scc}")

except Exception as e:
    print(f"Error al generar Gráfico 2: {e}")

# --- 4. Gráfico 3: Inercia Equivalente del Sistema ---
print("Generando Gráfico 3: Inercia del Sistema...")

try:
    plt.figure(figsize=(14, 7))
    # Creamos el 'ax' (ejes) aquí para que plt.savefig() funcione
    ax = df_freq.plot(x='time', y='inertia_equiv', kind='line', marker='s', label='Inercia Equivalente', color='green', linewidth=2, figsize=(14, 7))
    
    ax.set_title('Inercia Equivalente del Sistema por Hora', fontsize=16)
    ax.set_xlabel('Hora (time)', fontsize=12)
    ax.set_ylabel('Inercia (inertia_equiv) [MWs]', fontsize=12)
    ax.legend(fontsize=12)
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Ajustar los ticks del eje x
    times = df_freq['time']
    ax.set_xticks(ticks=times[::1])
    ax.set_xticklabels([str(t) for t in times[::1]])
    ax.set_xlim(times.min(), times.max())
    
    plt.tight_layout()
    
    plot_path_freq = os.path.join(output_dir, "inercia_sistema.png")
    # Usamos plt.savefig() sobre la figura actual que contiene 'ax'
    plt.savefig(plot_path_freq)
    plt.close()
    print(f"Gráfico 3 guardado en: {plot_path_freq}")

except Exception as e:
    print(f"Error al generar Gráfico 3: {e}")

# --- 5. Gráfico 4: RoCoF del Sistema ---
print("Generando Gráfico 4: RoCoF del Sistema...")

try:
    # Definir el límite de RoCoF
    rocof_limit = 0.5

    plt.figure(figsize=(14, 7))
    # Graficar los datos de RoCoF
    plt.plot(df_freq['time'], df_freq['rocof'], marker='x', linestyle='-', label='RoCoF del Sistema', color='purple', linewidth=2)
    
    # Dibujar la línea del límite
    plt.axhline(y=rocof_limit, color='red', linestyle='--', linewidth=2, label=f'Límite RoCoF = {rocof_limit} Hz/s')
    
    plt.title('RoCoF (Rate of Change of Frequency) del Sistema por Hora', fontsize=16)
    plt.xlabel('Hora (time)', fontsize=12)
    plt.ylabel('RoCoF [Hz/s]', fontsize=12)
    plt.legend(fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Ajustar los ticks del eje x
    times = df_freq['time']
    plt.xticks(ticks=times[::1], labels=[str(t) for t in times[::1]])
    plt.xlim(times.min(), times.max())
    
    # Asegurarnos de que el límite sea visible, ajustando el eje Y
    max_val = max(df_freq['rocof'].max(), rocof_limit)
    plt.ylim(0, max_val * 1.1) # Un 10% de margen superior
    
    plt.tight_layout()
    
    plot_path_rocof = os.path.join(output_dir, "rocof_sistema.png")
    plt.savefig(plot_path_rocof)
    plt.close()
    print(f"Gráfico 4 guardado en: {plot_path_rocof}")

except Exception as e:
    print(f"Error al generar Gráfico 4: {e}")


print("\n--- Proceso de graficación completado ---")

Cargando datos desde: modelo/data/output/resultados.xlsx
Guardando gráficos en: graficos_resultados/
Datos cargados exitosamente.
Generando Gráfico 1: Despacho de Generación...
Gráfico 1 guardado en: graficos_resultados\despacho_generacion.png
Generando Gráfico 2: SCC Mínima del Sistema...
Gráfico 2 guardado en: graficos_resultados\scc_minima_sistema.png
Generando Gráfico 3: Inercia del Sistema...
Gráfico 3 guardado en: graficos_resultados\inercia_sistema.png
Generando Gráfico 4: RoCoF del Sistema...
Gráfico 4 guardado en: graficos_resultados\rocof_sistema.png

--- Proceso de graficación completado ---


<Figure size 1400x700 with 0 Axes>

<Figure size 1400x700 with 0 Axes>